# Creating a dataframe with socioeconomic and electoral data (district-level)

In [1]:
# import libraries
import pickle
import pandas as pd
import numpy as np
import requests
import os

In [2]:
# set up working directory
os.path.abspath(os.getcwd()) # initial working directory (should be equal to source file directory if using Jupyter Notebook)
os.chdir('../../data/web_scraping') # change to directory where all data files are stored
# check working directory
os.path.abspath(os.getcwd())

'C:\\Users\\Simon\\OneDrive\\Uni\\LMU\\SS 2020\\Statistisches Consulting\\Bundestag-MP-Analyse\\data\\web_scraping'

### Socioeconomic data

In [6]:
# retrieve socioeconomic data from URL, save as df
url_strukturdaten = "https://www.bundeswahlleiter.de/dam/jcr/f7566722-a528-4b18-bea3-ea419371e300/btw17_strukturdaten.csv"
se_df = pd.read_csv(url_strukturdaten, encoding = "ISO-8859-1", delimiter = ';', decimal = ",")

In [7]:
se_df.columns = se_df.iloc[7] # setting header row
se_df = se_df.iloc[8:] # dropping non-meaningful rows
se_df = se_df.reset_index(drop=True) # reset index
se_df['Wahlkreis-Nr.'] = se_df['Wahlkreis-Nr.'].apply(lambda x: int(x)) # convert Wahlkreis-Nr. into integer

In [8]:
# filtering out NaN-rows and rows with state-wide total votes
mask1 = se_df[se_df['Wahlkreis-Nr.'] > 900].index
mask2 = se_df[se_df['Wahlkreis-Nr.'].isnull()].index
se_df = se_df.drop(mask1 | mask2)

In [9]:
# convert columns w/ numbers to numeric
non_numeric_columns = [0, 1, 2, len(se_df.columns) - 1]
numeric_columns = [element for element in range(len(se_df.columns)) if element not in non_numeric_columns]
for i in numeric_columns:
    se_df[se_df.columns[i]] = se_df[se_df.columns[i]].apply(lambda x: x.replace(',','.')).astype('float', errors = 'ignore')

In [10]:
# note: " - " is represented as square for column Wahlkreis-Name, coded as " \x96 "; need to address if column is to be used
se_df.head()

7,Land,Wahlkreis-Nr.,Wahlkreis-Name,Gemeinden am 31.12.2015 (Anzahl),Fläche am 31.12.2015 (km²),Bevölkerung am 31.12.2015 - Insgesamt (in 1000),Bevölkerung am 31.12.2015 - Deutsche (in 1000),Bevölkerung am 31.12.2015 - Ausländer (%),Bevölkerungsdichte am 31.12.2015 (Einwohner je km²),Zu- (+) bzw. Abnahme (-) der Bevölkerung 2015 - Geburtensaldo (je 1000 Einwohner),...,Sozialversicherungspflichtig Beschäftigte am 30.06.2016 - Übrige Dienstleister und 'ohne Angabe' (%),Empfänger(innen) von Leistungen nach SGB II am 31.12.2016 - insgesamt (je 1000 Einwohner),Empfänger(innen) von Leistungen nach SGB II am 31.12.2016 - nicht erwerbsfähige Hilfebedürftige (%),Empfänger(innen) von Leistungen nach SGB II am 31.12.2016 - Ausländer (%),Arbeitslosenquote März 2017 - insgesamt,Arbeitslosenquote März 2017 - Männer,Arbeitslosenquote März 2017 - Frauen,Arbeitslosenquote März 2017 - 15 bis unter 20 Jahre,Arbeitslosenquote März 2017 - 55 bis unter 65 Jahre,Fußnoten
0,Schleswig-Holstein,1,Flensburg  Schleswig,130.0,2128.1,282.8,266.7,5.7,132.9,-3.7,...,35.2,88.5,26.3,18.8,7.2,8.2,6.2,4.2,7.3,NaN
1,Schleswig-Holstein,2,Nordfriesland  Dithmarschen Nord,197.0,2777.0,232.3,219.7,5.4,83.6,-5.0,...,31.7,70.9,26.2,16.7,7.2,7.7,6.6,3.6,8.4,NaN
2,Schleswig-Holstein,3,Steinburg  Dithmarschen Süd,178.0,2000.5,220.8,209.8,5.0,110.4,-5.3,...,30.3,80.1,26.0,21.1,6.6,7.2,6.0,5.2,6.6,NaN
3,Schleswig-Holstein,4,Rendsburg-Eckernförde,163.0,2164.8,248.7,239.4,3.7,114.9,-3.4,...,34.7,59.4,28.1,23.8,5.1,5.5,4.7,3.2,5.6,NaN
4,Schleswig-Holstein,5,Kiel,3.0,143.0,268.0,242.9,9.4,1873.8,-0.1,...,38.6,138.7,26.4,28.3,8.8,10.0,7.6,6.1,8.7,NaN


### Election results (2017 federal elections)

In [11]:
url_zweitstimmen = "https://www.bundeswahlleiter.de/dam/jcr/72f186bb-aa56-47d3-b24c-6a46f5de22d0/btw17_kerg.csv"

In [12]:
zs_df = pd.read_csv(url_zweitstimmen, encoding = "ISO-8859-1", delimiter = ';', skiprows = 5)

In [13]:
zs_df = zs_df.iloc[2:] # dropping non-meaningful rows
zs_df = zs_df.reset_index(drop=True) # reset index
relevant_columns = [0, 1, 2, 17, 21, 25, 29, 33, 37, 41, 45] 
zs_df = zs_df.iloc[:, relevant_columns] # picking relevant columns
column_titles = ['Wahlkreis-Nr.', 'Bundesland', 'Bundesland-Nr.', 'Gesamt', 'CDU', 'SPD', 'Die Linke', 'Bündnis 90/Die Grünen', 'CSU', 'FDP', 'AFD']
zs_df.columns = column_titles

# convert columns w/ numbers to numeric
numeric_columns = ['Wahlkreis-Nr.', 'Bundesland-Nr.', 'Gesamt', 'CDU', 'SPD', 'Die Linke', 'Bündnis 90/Die Grünen', 'CSU', 'FDP', 'AFD']
for i in numeric_columns:
    zs_df[i] = pd.to_numeric(zs_df[i], errors = 'coerce')

In [14]:
# filtering out NaN-rows and rows with state-wide total votes
mask1 = zs_df[zs_df['Bundesland-Nr.'] == 99].index
mask2 = zs_df[zs_df['Bundesland-Nr.'].isnull()].index
zs_df = zs_df.drop(mask1 | mask2)

In [15]:
# adding voting shares
zs_df['CDU/CSU'] = np.where(zs_df['CSU'].isnull(), zs_df['CDU'], zs_df['CSU'])
zs_df['CDU/CSU Anteil'] = zs_df['CDU/CSU']/zs_df['Gesamt']
zs_df['SPD Anteil'] = zs_df['SPD']/zs_df['Gesamt']
zs_df['Die Linke Anteil'] = zs_df['Die Linke']/zs_df['Gesamt']
zs_df['Bündnis 90/Die Grünen Anteil'] = zs_df['Bündnis 90/Die Grünen']/zs_df['Gesamt']
zs_df['FDP Anteil'] = zs_df['FDP']/zs_df['Gesamt']
zs_df['AFD Anteil'] = zs_df['AFD']/zs_df['Gesamt']

In [16]:
zs_df.head()

,Wahlkreis-Nr.,Bundesland,Bundesland-Nr.,Gesamt,CDU,SPD,Die Linke,Bündnis 90/Die Grünen,CSU,FDP,AFD,CDU/CSU,CDU/CSU Anteil,SPD Anteil,Die Linke Anteil,Bündnis 90/Die Grünen Anteil,FDP Anteil,AFD Anteil
0,1.0,Flensburg â Schleswig,1.0,170465.0,58320.0,40388.0,14002.0,22304.0,NaN,18955.0,11653.0,58320.0,0.342123,0.236928,0.082140,0.130842,0.111196,0.068360
1,2.0,Nordfriesland â Dithmarschen Nord,1.0,138071.0,52928.0,31120.0,8589.0,15144.0,NaN,18050.0,9030.0,52928.0,0.383339,0.225391,0.062207,0.109683,0.130730,0.065401
2,3.0,Steinburg â Dithmarschen SÃ¼d,1.0,130878.0,47366.0,29756.0,8732.0,12960.0,NaN,17298.0,11180.0,47366.0,0.361910,0.227357,0.066719,0.099024,0.132169,0.085423
3,4.0,Rendsburg-EckernfÃ¶rde,1.0,156267.0,56585.0,35766.0,9962.0,19337.0,NaN,19071.0,11578.0,56585.0,0.362105,0.228877,0.063750,0.123743,0.122041,0.074091
4,5.0,Kiel,1.0,152069.0,40736.0,36208.0,15546.0,26143.0,NaN,17804.0,10504.0,40736.0,0.267878,0.238102,0.102230,0.171915,0.117078,0.069074


### Merging socioeconomic and election data

In [17]:
se_df = se_df.join(zs_df.set_index('Wahlkreis-Nr.'), on = 'Wahlkreis-Nr.')

In [18]:
se_df.head()

,Land,Wahlkreis-Nr.,Wahlkreis-Name,Gemeinden am 31.12.2015 (Anzahl),Fläche am 31.12.2015 (km²),Bevölkerung am 31.12.2015 - Insgesamt (in 1000),Bevölkerung am 31.12.2015 - Deutsche (in 1000),Bevölkerung am 31.12.2015 - Ausländer (%),Bevölkerungsdichte am 31.12.2015 (Einwohner je km²),Zu- (+) bzw. Abnahme (-) der Bevölkerung 2015 - Geburtensaldo (je 1000 Einwohner),...,CSU,FDP,AFD,CDU/CSU,CDU/CSU Anteil,SPD Anteil,Die Linke Anteil,Bündnis 90/Die Grünen Anteil,FDP Anteil,AFD Anteil
0,Schleswig-Holstein,1,Flensburg  Schleswig,130.0,2128.1,282.8,266.7,5.7,132.9,-3.7,...,NaN,18955.0,11653.0,58320.0,0.342123,0.236928,0.082140,0.130842,0.111196,0.068360
1,Schleswig-Holstein,2,Nordfriesland  Dithmarschen Nord,197.0,2777.0,232.3,219.7,5.4,83.6,-5.0,...,NaN,18050.0,9030.0,52928.0,0.383339,0.225391,0.062207,0.109683,0.130730,0.065401
2,Schleswig-Holstein,3,Steinburg  Dithmarschen Süd,178.0,2000.5,220.8,209.8,5.0,110.4,-5.3,...,NaN,17298.0,11180.0,47366.0,0.361910,0.227357,0.066719,0.099024,0.132169,0.085423
3,Schleswig-Holstein,4,Rendsburg-Eckernförde,163.0,2164.8,248.7,239.4,3.7,114.9,-3.4,...,NaN,19071.0,11578.0,56585.0,0.362105,0.228877,0.063750,0.123743,0.122041,0.074091
4,Schleswig-Holstein,5,Kiel,3.0,143.0,268.0,242.9,9.4,1873.8,-0.1,...,NaN,17804.0,10504.0,40736.0,0.267878,0.238102,0.102230,0.171915,0.117078,0.069074


In [159]:
# save output
with open('se_df.pickle', 'wb') as handle:
    pickle.dump(se_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [161]:
# # merging se_df into the main df (not doing it now, but later in R)

# with open('se_df.pickle', 'rb') as handle:
#     se_df = pickle.load(handle)
# with open('abg_df.pickle', 'rb') as handle:
#     df = pickle.load(handle)
# df = df.join(se_df.set_index('Wahlkreis-Nr.'), on = 'Wahlkreis-Nr.', rsuffix = '_se')
# df.head()

# with open('abg_df.pickle', 'wb') as handle:
#     pickle.dump(df, handle, protocol=pickle.HIGHEST_PROTOCOL)